In [7]:
# -*- coding: utf-8 -*-

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup as bs
import re, csv, time

next = ['Next'] #后面会用
url = 'http://www.agoda.com/pages/agoda/default/DestinationSearchResult.aspx?asq=LYqNjCU3K77lFCeHdsHoCcltLsM8gnXF9UdL4IY4sbxm2R%2fBEoUThw7YQMMd52jX64kk7a%2ftjPj1bp0uTDqoA4wRFe1oaY7p4FOxKNc8K55TMkun9NMX7y9sU13nDxwmbS3dzzacn%2bn7QZeTArmFDymZt%2bHTMhZofKWMifFALoJPCoQxAD18HHAgw7ya3ox%2b&city=3987&tick=635858641059&checkin=2015-12-17&checkout=2015-12-19&CheckInMonthYear=12-2015&los=2&SelectedGuestOption=2&rooms=1&adults=2&children=0&cityId=&countryId=&objectId=&pageTypeId=0&isHotel=False'

#def liking_level(soup): #
#    h3 = soup.select('.review-info h3')
#    fan = re.findall(r'"review-score">(.*?)</h3>', str(h3), re.S)
#    return fan

def level(soup, L1=[]):
    discount = soup.select('.hotel-info h3 i')
    star = re.findall(r'">(.*?)</i>', str(discount), re.S)
    L1 = [st.strip() for st in star]
    return L1

#def evaluate(soup):
#    eval_num = soup.select('.review-info a')
#    eval = re.findall(r'target="_blank">(.*?)</a>', str(eval_num), re.S)
#    return eval

def room_Type(soup):
    detail = soup.select('.room-type')
    room_type = re.findall(r'title="(.*?)">', str(detail), re.S)
    return room_type[::2]

def location(soup, L=None):
    locate = soup.select('.hotel-info strong a')
    lon = re.findall(r'Show map">(.*?)</a>', str(locate), re.S)
    for lo in lon:
        if L is None:
            L = []
        L.append(lo.strip())
    return L

def hotel_name(soup,L2=[]):
    hotel_name_field = soup.select('.hotel-info h3 a')
    L2 = [name.text for name in hotel_name_field]
    return L2

#下面的5行作用是去掉css js等，以加快浏览器加载速度
firefox_profile = webdriver.FirefoxProfile()
firefox_profile.set_preference("browser.download.folderList", 2)
firefox_profile.set_preference("permissions.default.stylesheet", 2)
firefox_profile.set_preference("permissions.default.image", 2)
firefox_profile.set_preference("javascript.enable", False)

browser = webdriver.Firefox(firefox_profile=firefox_profile)
#browser = webdriver.Firefox()
browser.implicitly_wait(10)
browser.get(url)
soup = bs(browser.page_source)

j = 1
while next == ['Next']:
    
#    Like = liking_level(soup)
#    Room_Type = room_Type(soup)
#    Evaluate = evaluate(soup)
    Level = level(soup)
    Hotle_Name = hotel_name(soup)
    Location = location(soup)
    
    with open('/home/mouse/anaconda3/agoda003.csv', 'a') as f:
        for i in range(len(Level)):
            hotelInfo = '%s; %s; %s' % (Hotle_Name[i], Level[i], Location[i])
            f.writelines(hotelInfo + '\n')
        print('Crawling...... \nThe page number is %d' %j)
    try:      
        pager_right = soup.select('.pager-right')
        next = re.findall(r'>(.*?)</a>', str(pager_right), re.S)
        browser.find_element_by_link_text("Next").click()
        time.sleep(3)
        soup = bs(browser.page_source)
        j += 1
    except NoSuchElementException:
        print('Done...........................')
    except StaleElementReferenceException:
        print('困死了，别问我哪里错了，吻我！！！！！！')
        
browser.close()

IndexError: list index out of range